In [ ]:
!pip install -U openpyxl selenium beautifulsoup4 lxml requests tika

In [ ]:
'''
注意事項:
下載對應的 ChromeDriver (web driver) 到程式檔案同一個目錄下後解壓縮，下載前記得對應版本編號。
連結: https://chromedriver.chromium.org/downloads

參考網頁:
[1] 成大歷史學報
https://his.ncku.edu.tw/p/412-1190-26290.php?Lang=zh-tw
[2] sqlite3 --- SQLite 数据库 DB-API 2.0 接口模块
https://docs.python.org/zh-tw/3/library/sqlite3.html
'''


'''
匯入套件
'''
# HTML parser
from bs4 import BeautifulSoup as bs

# 操作 browser 的 API
from selenium import webdriver

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 處理下拉式選單的工具
from selenium.webdriver.support.ui import Select

# 強制等待 (執行期間休息一下)
from time import sleep

# pretty-print
from pprint import pprint

# 隨機
from random import randint

# 計時
import time

# 整理 json 使用的工具
import json

# 執行 shell command 的時候用的
import os

# 取得錯誤訊息
import sys, traceback

# 子處理程序，用來取代 os.system 的功能
import subprocess

# 正規表達式
import re

# 編碼
from urllib.parse import quote

# SQLite 資料庫
import sqlite3

# 存取 Excel 的工具
from openpyxl import load_workbook
from openpyxl import Workbook

# 取得系統時間的工具
from datetime import datetime

# 引入 hashlib 模組
import hashlib

# 高階文件操作工具
import shutil

# 檔案剖析工具
from tika import parser

In [ ]:
# 啟動瀏覽器工具的選項
my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")             #不開啟實體瀏覽器背景執行
my_options.add_argument("--start-maximized")        #最大化視窗
my_options.add_argument("--incognito")              #開啟無痕模式
my_options.add_argument("--disable-popup-blocking") #禁用彈出攔截
my_options.add_argument("--disable-notifications")  #取消通知
my_options.add_argument("--lang=zh-TW")  #設定為正體中文
my_options.add_argument('--disable-gpu')
my_options.add_argument('--disable-software-rasterizer')
my_options.add_argument('--user-agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36"')

# 指定 chromedriver 檔案的路徑
driver_exec_path = './chromedriver.exe'

# 給 web driver 用的變數
driver = None

# 來源首頁
prefix_url = 'https://his.ncku.edu.tw'
url = prefix_url + '/p/412-1190-26290.php?Lang=zh-tw'

# 指定 sheet name
folderName = sheetName = 'his_ncku_edu_tw'

# 指定 json 檔名
jsonFileName = f'{folderName}.json'

# 建立儲存檔案用的資料夾
folderPath = f'./{folderName}'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)
    
# 設定 Chrome 下載路徑 (需要絕對路徑)
fullDownloadPath = os.getcwd() + '\\' + folderName

#預設下載路徑
my_options.add_experimental_option("prefs", {
    "download.default_directory": fullDownloadPath,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing_for_trusted_sources_enabled": False,
    "safebrowsing.enabled": False,
    "plugins.always_open_pdf_externally": True
})

# 放置爬取的資料
listData = []

In [ ]:
'''
函式
'''
# md5 (用來為每一筆資料建立唯一代號)
def md5(string):
    m = hashlib.md5()
    m.update(string.encode("utf-8"))
    return m.hexdigest()

# 初始化 Web Driver
def init():
    global driver, listData
    
    # 放置資料的變數初始化
    listData = []
    
    # 使用 Chrome 的 WebDriver
    driver = webdriver.Chrome( 
        options = my_options, 
        executable_path = driver_exec_path
    )
    
    
# 走訪頁面
def visit():
    global driver
    
    try:
        # 走訪首頁
        driver.get(url)

        # 等待目標元素出現
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, 'div#pageptlist')
            )
        )
        
    except TimeoutException as e:
        print('等待逾時: visit')
        
# 滾動頁面
def scroll():
    '''
    innerHeight => 瀏覽器內部的高度
    offset => 當前捲動的量(高度)
    count => 累計無效滾動次數
    limit => 最大無效滾動次數
    '''
    innerHeight = 0
    offset = 0
    count = 0
    limit = 3
    
    try:
        # 在捲動到沒有元素動態產生前，持續捲動
        while count <= limit:
            # 每次移動高度
            offset = driver.execute_script(
                'return window.document.documentElement.scrollHeight;'
            )

            '''
            或是每次只滾動一點距離，
            以免有些網站會在移動長距離後，
            將先前移動當中的元素隱藏

            例如將上方的 script 改成:
            offset += 600
            '''

            # 捲軸往下滑動
            driver.execute_script(f'''
                window.scrollTo({{
                    top: {offset}, 
                    behavior: 'smooth' 
                }});
            ''')

            # 強制等待，此時若有新元素生成，瀏覽器內部高度會自動增加
            sleep(1)

            # 透過執行 js 語法來取得捲動後的當前總高度
            innerHeight = driver.execute_script(
                'return window.document.documentElement.scrollHeight;'
            );

            # 經過計算，如果滾動距離(offset)大於等於視窗內部總高度(innerHeight)，代表已經到底了
            if offset == innerHeight:
                count += 1
            else:
                count = 0
    except SyntaxError:
        print('語法錯誤: scroll')
        
# 剖析元素資料
def parse():
    global driver, listData
    try:
        # 等待目標元素出現
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, 'div#pageptlist div.listBS div.mtitle a')
            )
        )
        
        # 取得各期文章 
        for a in driver.find_elements(By.CSS_SELECTOR, 'div#pageptlist div.listBS div.mtitle a'):
            # 刊號
            publish_num = a.get_attribute('innerText').strip()
            
            # 內頁連結
            link = a.get_attribute('href')
            
            # 整理資料
            listData.append({
                'journal_title': '成大歷史學報',
                'publish_num': publish_num,
                'link': link
            })
            
        # 取得內頁中所有 pdf 資訊
        for index, myDict in enumerate(listData):
            # 如果沒有內頁資料，則另外新增
            if not 'sub' in listData[index]:
                listData[index]['sub'] = []
            
            # 走訪內頁
            driver.get(myDict['link'])
            
            # 等待目標元素出現
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located(
                    (By.CSS_SELECTOR, 'ul.mptattach li a')
                )
            )
            
            # 強制停止
            sleep(1)
            
            # 取得 pdf 資訊
            for a in driver.find_elements(By.CSS_SELECTOR, 'ul.mptattach li a'):
                # 篇名
                journal_sub_title = a.get_attribute('innerText').strip()
                
                # pdf 連結
                pdf_link = a.get_attribute('href')
                
                # 整理資料
                listData[index]['sub'].append({
                    'id': md5(pdf_link),
                    'journal_sub_title': journal_sub_title,
                    'pdf_link': pdf_link
                })

    except TimeoutException as e:
        print('等待逾時: parse')
        
# 取得出版日期
def getPublishDate():
    global listData
    
    # 過濾出版日期
    regexPublishDate = r'\d{4}\s?年\s?\d{1,2}\s?月'

    # 嘗試取得出版日期
    for index, myDict in enumerate(listData):
        for idx, d in enumerate(listData[index]['sub']):
            # 出版日期
            listData[index]['sub'][idx]['publish_date'] = ''

            # 開啟 pdf 檔案，取得出版日期
            content = parser.from_file(d["pdf_link"])['content']
            if content != None:
                matchPublishDate = re.search(regexPublishDate, content)
                if matchPublishDate != None:
                    # 整理資料
                    listData[index]['sub'][idx]['publish_date'] = matchPublishDate[0]

# 關閉瀏覽器
def close():
    global driver
    driver.quit()
        
# 儲存成 json
def saveJson():
    global listData
    with open(f"{folderPath}/{jsonFileName}", "w", encoding="utf-8") as file:
        file.write( json.dumps( listData, ensure_ascii=False, indent=4 ) )
    
# 儲存 .db
def saveDB():
    with open(f"{folderPath}/{jsonFileName}", "r", encoding="utf-8") as file:      
        # 取得 json 內容
        strJson = file.read()
        
        # 將 json 轉成 list
        listJson = json.loads(strJson)
        
    # 寫入對話記錄
    conn = sqlite3.connect(f"{folderPath}/{folderName}.db")
    
    # 建立 cursor 物件
    cursor = conn.cursor()

    # 執行 SQL 語法
    try:
        # 查詢特定資料，看看是否已經存在於資料表當中
        sql_query = f'''
        SELECT 1
        FROM journals
        WHERE id = ?
        '''
        
        # 寫入資料
        sql_insert = f'''
        INSERT INTO journals (
            id, journal_title, publish_num, publish_date, journal_sub_title, 
            link, pdf_link, is_downloaded, created_at, updated_at
        ) VALUES ( 
            ?,?,?,?,?,
            ?,?,?,?,?
        )
        '''
        
        # 放置準備寫入的資料
        list_insert = []
        
        # 將 json 資料一筆一筆找出來
        for myDict in listJson:
            # 每一筆資料裡面有 sub 屬性，裡面有內頁的 pdf 連結等資訊，所以也要找出來
            for d in myDict['sub']:
                # 如果資料庫沒有這筆資料(透過 id)，則暫時將資料以 tuple 格式放到 list 當中
                if cursor.execute(sql_query, (d["id"],)).fetchone() == None:
                    list_insert.append((
                        d['id'],
                        myDict['journal_title'],
                        myDict['publish_num'],
                        d['publish_date'],
                        d['journal_sub_title'],
                        myDict['link'],
                        d['pdf_link'],   
                        0,
                        datetime.today().strftime("%Y-%m-%d %H-%M-%S"),
                        datetime.today().strftime("%Y-%m-%d %H-%M-%S")
                    ))
        
        # 新增資料到資料庫當中
        cursor.executemany(sql_insert, list_insert)
        
        # 執行 SQL 語法
        conn.commit()
    except sqlite3.Error as err: 
        # 回滾
        conn.rollback()

        # SQLite3 例外處理
        exc_type, exc_value, exc_tb = sys.exc_info()
        strErrorMsg = f'''SQLite error: {' '.join(err.args)}\n\n
        SQLite traceback: {traceback.format_exception(exc_type, exc_value, exc_tb)}
        '''
        print(strErrorMsg)
    finally:
        # 關閉 sqlite
        conn.close()
    
# 下載
def download():
    # 寫入對話記錄
    conn = sqlite3.connect(f"{folderPath}/{folderName}.db")
    
    # 將查詢出來的結果 (tuple)，變成 key-value 型式 (dict)
    conn.row_factory = sqlite3.Row
    
    # 建立 cursor 物件
    cursor = conn.cursor()

    # 執行 SQL 語法
    try:
        # 查詢尚未下載的資料
        sql_query = f'''
        SELECT sn, id, pdf_link
        FROM journals
        WHERE `is_downloaded` = 0
        '''
        
        # 更新資料的欄位(是否下載過)
        sql_update = f'''
        UPDATE `journals` 
        SET 
            `is_downloaded` = 1 ,
            `updated_at` = ?
        WHERE `id` = ?
        '''
            
        # 取得所有未下載的資料
        for myDict in cursor.execute(sql_query).fetchall():
            # 等待
            sleep(randint(1,2))

            # 下載 pdf
            cmd = [
                'curl', 
                '-H', 'User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36',
                '-L', myDict["pdf_link"], 
                '-o', f'{folderPath}/{myDict["sn"]}_{myDict["id"]}.pdf'
            ]
            result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
            #output = result.stdout
            #pprint(output)
            print(f'{folderPath}/{myDict["sn"]}_{myDict["id"]}.pdf')

            # 將 is_downloaded 改成 1，代表已下載過
            cursor.execute(sql_update, (datetime.today().strftime("%Y-%m-%d %H-%M-%S"), myDict["id"],))
                
        conn.commit()
    except sqlite3.Error as err: 
        # 回滾
        conn.rollback()

        # SQLite3 例外處理
        exc_type, exc_value, exc_tb = sys.exc_info()
        strErrorMsg = f'''SQLite error: {' '.join(err.args)}\n\n
        SQLite traceback: {traceback.format_exception(exc_type, exc_value, exc_tb)}
        '''
        print(strErrorMsg)
    finally:
        # 關閉 sqlite
        conn.close()

In [ ]:
init()

In [ ]:
visit()

In [ ]:
scroll()

In [ ]:
parse()

In [ ]:
time_begin = time.time()
getPublishDate()
time_end = time.time()
print(f"總共執行了 { time_end - time_begin } 秒")
# 總共執行了 533.356217622757 秒

In [ ]:
close()

In [ ]:
saveJson()

In [ ]:
saveDB()

In [ ]:
time_begin = time.time()
download()
time_end = time.time()
print(f"總共執行了 { time_end - time_begin } 秒")
# 總共執行了 1032.4337289333344 秒